In [18]:
#GoogLeNet
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn

class Inception(nn.Block):
    # c1 - c4为每条线路⾥的层的输出通道数
    def __init__(self,c1,c2,c3,c4,**kwargs):
        super(Inception,self).__init__(**kwargs)
        #线路1
        self.p1_1 = nn.Conv2D(c1,kernel_size=1,activation='relu')
        #线路2
        self.p2_1 = nn.Conv2D(c2[0],kernel_size=1,activation='relu')
        self.p2_2 = nn.Conv2D(c2[1],kernel_size=3,padding =1, activation='relu')
        #线路3
        self.p3_1 = nn.Conv2D(c3[0],kernel_size=1,activation='relu')
        self.p3_2 = nn.Conv2D(c3[1],kernel_size=5,padding=2,activation='relu')
        #线路4
        self.p4_1 = nn.MaxPool2D(pool_size=3,strides=1,padding=1)
        self.p4_2 = nn.Conv2D(c4,kernel_size=1,activation='relu')
        
    def forward(self,x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return nd.concat(p1,p2,p3,p4,dim=1)# 在通道维上连结输出


In [19]:
b1 = nn.Sequential()
b1.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3,activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [20]:
b2 = nn.Sequential()
b2.add(nn.Conv2D(64, kernel_size=1, activation='relu'),
        nn.Conv2D(192, kernel_size=3, padding=1, activation='relu'),
        nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [21]:
b3 = nn.Sequential()
b3.add(Inception(64, (96, 128), (16, 32), 32),
    Inception(128, (128, 192), (32, 96), 64),
    nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [22]:
b4 = nn.Sequential()
b4.add(Inception(192, (96, 208), (16, 48), 64),
    Inception(160, (112, 224), (24, 64), 64),
    Inception(128, (128, 256), (24, 64), 64),
    Inception(112, (144, 288), (32, 64), 64),
    Inception(256, (160, 320), (32, 128), 128),
    nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [23]:
b5 = nn.Sequential()
b5.add(Inception(256, (160, 320), (32, 128), 128),
       Inception(384, (192, 384), (48, 128), 128),
        nn.GlobalAvgPool2D())
net = nn.Sequential()
net.add(b1,b2,b3,b4,b5,nn.Dense(10))

In [24]:
x = nd.random.uniform(shape=(1,1,96,96))
net.initialize()
for layer in net:
    x = layer(x)
    print(layer.name, 'output shape:\t', x.shape)


sequential13 output shape:	 (1, 64, 24, 24)
sequential14 output shape:	 (1, 192, 12, 12)
sequential15 output shape:	 (1, 480, 6, 6)
sequential16 output shape:	 (1, 832, 3, 3)
sequential17 output shape:	 (1, 1024, 1, 1)
dense2 output shape:	 (1, 10)


In [ ]:
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
    num_epochs)
